In [ ]:
from langchain_community.chat_models import ChatZhipuAI

In [ ]:
ZHIPUAI_API_KEY = "0f2f6aca4fb384e90505b066a1ddf204.hM6JYEdsgvTYLxWz"
llm = ChatZhipuAI(
    temperature=0.1,
    api_key=ZHIPUAI_API_KEY,
    model_name="glm-4",
)

In [ ]:
# !pip install pypdf

In [ ]:

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path='初赛训练数据集.pdf')
data = loader.load()

In [ ]:
from pprint import pprint
pprint(data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=40, chunk_overlap=10, add_start_index=True
)
all_splits = text_splitter.split_documents(data)
len(all_splits)


In [29]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bgeEmbeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(all_splits, bgeEmbeddings)


g:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [30]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever.invoke("问题：在国家环保法要求下，哪些情况下需要对车辆进行报废处理？")


[Document(page_content='辆应按国家环保法规要求进行报废处理。报废车辆前，请联系Lynk&Co领克中心，咨', metadata={'source': '初赛训练数据集.pdf', 'page': 13, 'start_index': 278}),
 Document(page_content='按照国家环保法要求，达到使用年限或不能满足道路使用条件的车辆应按国家环保法规要', metadata={'source': '初赛训练数据集.pdf', 'page': 13, 'start_index': 249}),
 Document(page_content='■报废处理时必须遵守有关的安全条例与法律法规。\n隐私告知', metadata={'source': '初赛训练数据集.pdf', 'page': 13, 'start_index': 360})]

In [31]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""仅根据所提供的上下文回答以下问题:

<context>
{context}
</context>

问题: {question}""")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
retriever_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
retriever_chain.invoke("问题：LynkCo-Pilot系统在何种情况下会默认将跟车距离间隔设置为最远？")


'根据所提供的上下文，LynkCo-Pilot系统在每次启动车辆时会默认将跟车距离间隔设置为最远。具体的描述出现在文档中：“跟车距离间隔越长。每次启动车辆时，LynkCo-Pilot默认跟车距离间隔越长。” 这意味着为了安全起见，当驾驶员启动车辆后，LynkCo-Pilot系统会预设一个较长的跟车距离间隔。'